# Semestrální práce KMA/NA

Jan Půlpán

## Úkol 8.

In [1]:
import numpy as np
import numpy.linalg as la
import scipy.io as sio

#nahrání matice ze souboru
mat = sio.loadmat('zenios.mat')
A = mat['A'].astype('float64')
print(f'Řád matice A: {A.shape}')

Řád matice A: (2873, 2873)


### 8. Numerická hodnost matice

Matici $A$ jsme načetli z .mat Matlabovského souboru `zenios.mat` a udělali konverzi na typ `float64`, aby při výpočtech byla použita double precision.

Nejprve vypočteme singulární rozklad matice $A$ pro který platí $$A = U \Sigma V^*.$$

In [2]:
%%time
U, s, Vh = la.svd(A)

CPU times: user 21.8 s, sys: 661 ms, total: 22.4 s
Wall time: 12.7 s


Numerickou hodnost matice $A$ určíme jako počet singulárních čísel výrazně větších než strojová přesnost. V našem případě tedy počet singulárních čísel pro která platí $\sigma_i > nu \Vert A \Vert $, kde $n$ je řád matice $A$ a $u \approx 2.2 \times 10^{-16}$.

In [3]:
u = 2.2e-16
n, _ = A.shape
A_norm = la.norm(A,ord=2)

tol = n*u*A_norm

# numerická hodnost --> počet prvků větších než tol
r_num = (s > tol).sum()
print(f'Numerická hodnost: {r_num}')

Numerická hodnost: 207


In [4]:
s[199:209]

array([5.10395975e-06, 4.37853039e-06, 3.56941996e-06, 1.74319027e-06,
       1.21854379e-06, 1.09753424e-06, 1.67848833e-10, 1.18870819e-11,
       1.01321056e-15, 6.99494910e-16])

In [7]:
s[205]-tol

1.6640675809301832e-10

In [8]:
tol

1.4420748188502917e-12

Numerickou hodnost vypočtenou přes singulární čísla porovnáme s hodností matice vypočtené pomocí funkce `numpy.linalg.matrix_rank()`. Výsledky jsou shodné, i díky tomu že Numpy počítá hodnost matice také přes singulární rozklad ([viz dokumentace](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.matrix_rank.html)).

In [4]:
A_rank = la.matrix_rank(A)
print(f'Hodnost vypočtená přes matrix_rank(): {A_rank} ')

Hodnost vypočtená přes matrix_rank(): 207 


Nyní nás zajímá velikost $\Vert A - A_k \Vert$, kterou určíme jen pomocí singulárních čísel. $A_k$ je nejlepší aproximace matice $A$ s hodností $k$. Tu získáme pomocí singulárního rozkladu $A = U \Sigma V^*$ vztahem $$ 
A_k = \sum_{i=1}^k \sigma_j u_j v_j^*, \quad k < r, \quad r = \textrm{rank}(A).$$
Navíc platí $\Vert A - A_k \Vert = \sigma_{k+1}, \, k < r$, kde $r$ je vypočtená numerická hodnost.

Rádi bychom určili velikost normy $\Vert A - A_r \Vert$ jen pomocí singulárního čísla. Podle předchozího musí ale platit $k < r$, proto najdeme výsledek pro $\Vert A - A_{r-1} \Vert$.

In [9]:
print(f'Velikost ||A-Ar-1||: {s[r_num]}')

Velikost ||A-Ar-1||: 1.0132105604656179e-15


Nyní ověříme přes numerický výpočet normy $\Vert A - A_{r-1} \Vert$.

In [10]:
A_k = U[:,:r_num] @ np.diag(s[:r_num]) @ Vh[:r_num,:]
print(f'Velikost ||A-Ar-1|| numericky: {la.norm(A-A_k, ord=2)}')

Velikost ||A-Ar-1|| numericky: 1.6241447208991068e-14


In [ ]:
o